In [1]:
%cd ..

/home/ubuntu/dev/reid/pepper


In [2]:
import os.path as osp
from copy import deepcopy

import numpy as np
import torch

import mmcv

# Check market1501 evaluation

- [x] load config
- [x] load pretrained model
- [x] build dataset
- [x] build a validation dataloader
- [ ] get features

Debugging:
- figure out why the results are so low

In [3]:
# load configs
from mmcv import Config

_root = 'configs'
_model = 'resnet'
_param = 'resnet50_b32'
_dataset = 'market1501'
_cfg_name = f'{_param}_{_dataset}.py'

cfg_path = osp.join(_root, _model, _cfg_name)
assert osp.exists(cfg_path), f"{cfg_path} doesn't exist"

cfg = Config.fromfile(cfg_path)
print(cfg.pretty_text)

model = dict(
    type='ImageReID',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    neck=dict(type='KernelGlobalAveragePooling', kernel_size=(8, 4), stride=1),
    head=dict(
        type='LinearReIDHead',
        num_fcs=1,
        in_channels=2048,
        fc_channels=1024,
        out_channels=128,
        num_classes=380,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        loss_pairwise=dict(type='TripletLoss', margin=0.3, loss_weight=1.0),
        norm_cfg=dict(type='BN1d'),
        act_cfg=dict(type='ReLU')),
    init_cfg=dict(
        type='Pretrained',
        checkpoint=
        'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'
    ))
sampler = dict(
    type='BalancedIdentityDistributedSampler',
    batch_size=32,
    num_instances=4,
    shuffle=True)
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std

In [7]:
# initialize dataset
from pepper.datasets import build_dataset, build_dataloader, replace_ImageToTensor

# load validation as dataset
val_samples_per_gpu = 1
if val_samples_per_gpu > 1:
    # Replace 'ImageToTensor' to 'DefaultFormatBundle'
    cfg.data.val.pipeline = replace_ImageToTensor(cfg.data.val.pipeline)

dataset = build_dataset(cfg.data.val, dict(eval_mode=True))

dataloader = build_dataloader(
    dataset,
    samples_per_gpu=val_samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False,
    round_up=True,
    is_val=True,
)

In [8]:
# initialize model and load pretrained model
from mmcv.runner.checkpoint import load_checkpoint
from pepper.models import build_reid


_pretrain_root = osp.join('work_dirs', f'{_param}_{_dataset}')
assert osp.exists(_pretrain_root), f"{_pretrain_root} doesn't exist"

_pretrain_pth = "latest.pth"
_pretrain_path = osp.join(_pretrain_root, _pretrain_pth)
assert osp.exists(_pretrain_path), f"{_pretrain_path} doesn't exist"

cfg.model.head.num_classes = dataset.NUM_PIDS
cfg.model.head.num_classes = 751
if cfg.get("train_cfg", False):
    model = build_reid(
        cfg.model,
        train_cfg=cfg.train_cfg,
        test_cfg=cfg.test_cfg,
    )
else:
    model = build_reid(cfg.model)

# model.init_weights()
load_checkpoint(model, _pretrain_path)

# print(model)

model = model.cuda()

print("done")

load checkpoint from local path: work_dirs/resnet50_b32_market1501/latest.pth
done


In [9]:
# compute features for validation dataset

def get_features(_model, _dataloader):
    prog_bar = mmcv.ProgressBar(len(_dataloader.dataset))
    
    feats = []
    for i, data in enumerate(_dataloader):
        img = data['img'].cuda()
        with torch.no_grad():
            feat = model(img=img, return_loss=False)

        batch_size = len(feat)
        feats.extend(feat)
        
        for _ in range(batch_size):
            prog_bar.update()
            
    return feats

model.eval()

results = get_features(model, dataloader)

[>>>>>>>>>>>>>>>>>>>>>>>] 19281/19281, 161.7 task/s, elapsed: 119s, ETA:     0s

In [12]:
print(len(results))

19281


In [13]:
# evaluate
from pepper.core.evaluation import evaluate


def compute_metrics(results, dataset):
    
    results = [result.data.cpu().squeeze() for result in results]
    features = torch.stack(results)
    pids = dataset.get_pids()
    camids = dataset.get_camids()
    
    assert len(features) == len(dataset.data_infos)
    q_feat = features[: dataset._num_query]
    g_feat = features[dataset._num_query :]
    q_pids = pids[: dataset._num_query]
    g_pids = pids[dataset._num_query :]
    q_camids = camids[: dataset._num_query]
    g_camids = camids[dataset._num_query :]
    
    print(features.shape)
    print('q_feat', q_feat.shape)
    print('g_feat', g_feat.shape)
    print('q_pids', q_pids.shape)
    print('g_pids', g_pids.shape)
    print('q_camids', q_camids.shape)
    print('g_camids', g_camids.shape)

    eval_results = evaluate(
        q_feat=q_feat,
        g_feat=g_feat,
        q_pids=q_pids,
        g_pids=g_pids,
        q_camids=q_camids,
        g_camids=g_camids,
        metric="euclidean",
        ranks=[1, 5, 10, 20],
        use_metric_cuhk03=False,
        use_aqe=False,
        qe_times=1,
        qe_k=5,
        alpha=3.0,
        rerank=False,
        k1=20,
        k2=6,
        lambda_value=0.3,
        use_roc=False,
    )
    
    return eval_results


eval_results = compute_metrics(results, dataloader.dataset)

torch.Size([19281, 128])
q_feat torch.Size([3368, 128])
g_feat torch.Size([15913, 128])
q_pids (3368,)
g_pids (15913,)
q_camids (3368,)
g_camids (15913,)


In [11]:
eval_results

OrderedDict([('Rank-1', 0.8358076),
             ('Rank-5', 0.93824226),
             ('Rank-10', 0.9599169),
             ('Rank-20', 0.976247),
             ('CMC',
              array([0.8358076 , 0.8937055 , 0.9171615 , 0.9314133 , 0.93824226,
                     0.9447743 , 0.95071256, 0.95427555, 0.95665085, 0.9599169 ,
                     0.96229213, 0.9652613 , 0.9679335 , 0.96941805, 0.9709026 ,
                     0.97149646, 0.9726841 , 0.9738717 , 0.9747625 , 0.976247  ,
                     0.97713774, 0.9780285 , 0.9786223 , 0.97921616, 0.97951305,
                     0.97951305, 0.97981   , 0.9801069 , 0.98070073, 0.9809976 ,
                     0.9809976 , 0.98188835, 0.9824822 , 0.98307604, 0.9833729 ,
                     0.9833729 , 0.9833729 , 0.9836698 , 0.98396677, 0.98426366,
                     0.98456055, 0.98456055, 0.9848575 , 0.9854513 , 0.9854513 ,
                     0.9854513 , 0.9854513 , 0.98574823, 0.98663896, 0.98693585],
                    dt